<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Графики.-Уточнить-легенду!" data-toc-modified-id="Графики.-Уточнить-легенду!-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Графики. Уточнить легенду!</a></span></li></ul></div>

In [1]:
import pandas as pd
import bokeh
from bokeh.plotting import figure, show, save, output_file
from bokeh.models import ColumnDataSource
from bokeh.io import export_png
from bokeh.models import Legend,LegendItem
import math
from bokeh.models import Label
from tqdm.notebook import tqdm as tqdm
import time

import sys
sys.path.insert(0, '../') 
from btb_matrix.domain import FullBtBways
from btb_matrix.config import engine

In [2]:
current = FullBtBways('2022-06-08', 194, start_time = '7:30')
base = FullBtBways('2022-06-08', 1, start_time = '7:30')

C:\Users\ovnanyan_gm\AppData\Local\anaconda3\lib\site-packages\bcolz\ctable.py:806: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  df = pd.DataFrame.from_items(


..\data\processed\2022-06-08\0730_0829_scenario_194\costs\oneride_costs.bcolz loaded from cache.
..\data\processed\2022-06-08\0730_0829_scenario_1\costs\oneride_costs.bcolz loaded from cache.


In [3]:
matrix_current = current.get_trip_flows()

# Добавляев название о.п.
stops = pd.read_sql('select stop_id, stop_name from routes.stops()', con=engine)
stops = stops.set_index('stop_id')
matrix_current = matrix_current.join(stops, on='stop_id_start')
matrix_current = matrix_current.join(stops, on='stop_id_end', lsuffix='_start', rsuffix='_end')
del stops

# Добавляем уровень загрузки
matrix_current['cnt_inside'] = matrix_current['cnt_inside'] 
matrix_current['cnt_on'] = matrix_current['cnt_on']
matrix_current['cnt_off'] = matrix_current['cnt_off']

# Добавляем уровень загрузки
matrix_current['rel_inside'] = matrix_current['cnt_inside'] / matrix_current['capacity']

matrix_current.head(3)

..\data\processed\2022-06-08\0730_0829_scenario_194\flows\trip_flows.bcolz loaded from cache.


,route_id,route_name,variant_id,variant_name,trip_id,mvn,segment_seq,stop_id_start,length,stop_id_end,site_id_start,site_id_end,num_of_trips,capacity,cnt_on,cnt_inside,cnt_off,stop_name_start,stop_name_end,rel_inside
0,103,А_127,204520,127-У1-А,64216,127-У1-А-прямое,1,10318,0.112635,5019,21,21,2.0,214.0,0.000000,0.000000,0.0,Рублево,Рублево,0.000000
1,103,А_127,204520,127-У1-А,64216,127-У1-А-прямое,2,5019,0.741055,5020,21,22,2.0,214.0,17.763745,17.763745,0.0,Рублево,Советская ул.,0.083008
2,103,А_127,204520,127-У1-А,64216,127-У1-А-прямое,3,5020,0.607082,8749,22,23,2.0,214.0,11.967583,29.731328,0.0,Советская ул.,Поворот на Рублево,0.138931


In [4]:
matrix_base = base.get_trip_flows()

# Добавляев название о.п.
stops = pd.read_sql('select stop_id, stop_name from routes.stops()', con=engine)
stops = stops.set_index('stop_id')
matrix_base = matrix_base.join(stops, on='stop_id_start')
matrix_base = matrix_base.join(stops, on='stop_id_end', lsuffix='_start', rsuffix='_end')


# Добавляем уровень загрузки
matrix_base['cnt_inside'] = matrix_base['cnt_inside'] 
matrix_base['cnt_on'] = matrix_base['cnt_on'] 
matrix_base['cnt_off'] = matrix_base['cnt_off'] 

# Добавляем уровень загрузки
matrix_base['rel_inside'] = matrix_base['cnt_inside'] / matrix_base['capacity']

matrix_base.head(3)

..\data\processed\2022-06-08\0730_0829_scenario_1\flows\trip_flows.bcolz loaded from cache.


,route_id,route_name,variant_id,variant_name,trip_id,mvn,segment_seq,stop_id_start,length,stop_id_end,site_id_start,site_id_end,num_of_trips,capacity,cnt_on,cnt_inside,cnt_off,stop_name_start,stop_name_end,rel_inside
0,103,А_127,204520,127-У1-А,64216,127-У1-А-прямое,1,10318,0.112635,5019,21,21,2.0,214.0,0.000000,0.000000,0.0,Рублево,Рублево,0.000000
1,103,А_127,204520,127-У1-А,64216,127-У1-А-прямое,2,5019,0.741055,5020,21,22,2.0,214.0,17.763745,17.763745,0.0,Рублево,Советская ул.,0.083008
2,103,А_127,204520,127-У1-А,64216,127-У1-А-прямое,3,5020,0.607082,8749,22,23,2.0,214.0,11.967583,29.731328,0.0,Советская ул.,Поворот на Рублево,0.138931


In [5]:
# файл с остановками МЦД/БКЛ, чтобы отметить на графике
stops_metro = pd.read_excel('stops0.xlsx',  engine='openpyxl', encoding = 'cp1251')
stops_metro = list(stops_metro['stop_id'])
stops_metro[:3]

[nan, nan, nan]

In [6]:
# приводим в нужный вид

matrix_current_inputs = matrix_current[['mvn','route_id','segment_seq','stop_id_start','cnt_on','capacity','num_of_trips', 'cnt_inside']]
matrix_current_inputs = matrix_current_inputs.rename(columns={'segment_seq':'stop_sequence',
                                'stop_id_start':'stop_id'})


matrix_current_outputs = matrix_current[['mvn','route_id','stop_id_end','cnt_off']]
matrix_current_outputs = matrix_current_outputs.rename(columns={'stop_id_end':'stop_id'})


mtrx = pd.merge(matrix_current_inputs, matrix_current_outputs, how='outer',on=['mvn','route_id','stop_id'])
mtrx = mtrx.sort_values(['mvn','route_id','stop_sequence','stop_id']).reset_index(drop=True)
mtrx = mtrx.fillna(0)

matrix_current = mtrx.copy()
matrix_current = pd.merge(matrix_current, stops, how='left', on = ['stop_id'])

matrix_current.head()

,mvn,route_id,stop_sequence,stop_id,cnt_on,capacity,num_of_trips,cnt_inside,cnt_off,stop_name
0,1-00-А-обратное,247,1.0,6227,2.685040,136.0,2.0,4.109201,0.000000,Севастопольский просп.
1,1-00-А-обратное,247,2.0,1002957,3.986380,136.0,2.0,8.095581,0.000000,"Ул. Обручева, 45"
2,1-00-А-обратное,247,3.0,3496,0.000000,136.0,2.0,7.989453,0.106128,Ул. Введенского
3,1-00-А-обратное,247,4.0,3497,2.948064,136.0,2.0,10.937516,0.000000,З-д спецсплавов
4,1-00-А-обратное,247,5.0,3498,0.166891,136.0,2.0,9.404005,1.700402,"Ул. Обручева, 29"


In [7]:
# приводим в нужный вид

matrix_base_inputs = matrix_base[['mvn','route_id','segment_seq','stop_id_start','cnt_on','capacity','num_of_trips', 'cnt_inside']]
matrix_base_inputs = matrix_base_inputs.rename(columns={'segment_seq':'stop_sequence',
                                'stop_id_start':'stop_id'})


matrix_base_outputs = matrix_base[['mvn','route_id','stop_id_end','cnt_off']]
matrix_base_outputs = matrix_base_outputs.rename(columns={'stop_id_end':'stop_id'})


mtrx = pd.merge(matrix_base_inputs, matrix_base_outputs, how='outer',on=['mvn','route_id','stop_id'])
mtrx = mtrx.sort_values(['mvn','route_id','stop_sequence','stop_id']).reset_index(drop=True)
mtrx = mtrx.fillna(0)

matrix_base = mtrx.copy()
matrix_base = pd.merge(matrix_base, stops, how='left', on = ['stop_id'])

matrix_base.head()

,mvn,route_id,stop_sequence,stop_id,cnt_on,capacity,num_of_trips,cnt_inside,cnt_off,stop_name
0,1-00-А-обратное,247,1.0,6227,2.685040,136.0,2.0,4.109201,0.000000,Севастопольский просп.
1,1-00-А-обратное,247,2.0,1002957,3.986380,136.0,2.0,8.095581,0.000000,"Ул. Обручева, 45"
2,1-00-А-обратное,247,3.0,3496,0.000000,136.0,2.0,7.989453,0.106128,Ул. Введенского
3,1-00-А-обратное,247,4.0,3497,2.948064,136.0,2.0,10.937516,0.000000,З-д спецсплавов
4,1-00-А-обратное,247,5.0,3498,0.166891,136.0,2.0,9.404005,1.700402,"Ул. Обручева, 29"


In [8]:
matrix_all = pd.merge(matrix_current,matrix_base,how='left',on=['mvn','route_id','stop_id', 'stop_sequence'])
matrix_all = matrix_all.rename(columns={'cnt_on_x':'cnt_on_current','cnt_off_x':'cnt_off_current',
                                       'cnt_inside_x':'cnt_inside_current','cnt_on_y':'cnt_on_base',
                                        'cnt_off_y':'cnt_off_base','cnt_inside_y':'cnt_inside_base',
                                       'capacity_x':'capacity', 'num_of_trips_x':'num_of_trips', 'stop_name_x' : 'stop_name'})
matrix_all = matrix_all.drop(columns=['capacity_y','num_of_trips_y', 'stop_name_y'])
matrix_all.head()

,mvn,route_id,stop_sequence,stop_id,cnt_on_current,capacity,num_of_trips,cnt_inside_current,cnt_off_current,stop_name,cnt_on_base,cnt_inside_base,cnt_off_base
0,1-00-А-обратное,247,1.0,6227,2.685040,136.0,2.0,4.109201,0.000000,Севастопольский просп.,2.685040,4.109201,0.000000
1,1-00-А-обратное,247,2.0,1002957,3.986380,136.0,2.0,8.095581,0.000000,"Ул. Обручева, 45",3.986380,8.095581,0.000000
2,1-00-А-обратное,247,3.0,3496,0.000000,136.0,2.0,7.989453,0.106128,Ул. Введенского,0.000000,7.989453,0.106128
3,1-00-А-обратное,247,4.0,3497,2.948064,136.0,2.0,10.937516,0.000000,З-д спецсплавов,2.948064,10.937516,0.000000
4,1-00-А-обратное,247,5.0,3498,0.166891,136.0,2.0,9.404005,1.700402,"Ул. Обручева, 29",0.166891,9.404005,1.700402


In [9]:
# МВН для графика
# mvns = [
# '384-00-А-прямое',
# '384-00-А-обратное',
# 'т29-00-А-прямое',
# 'т29-00-А-обратное',
# 'т42-00-А-прямое',
# 'т42-00-А-обратное',
# 'с484-00-А-прямое',
# 'с484-00-А-обратное'
# ]


# выгрузка всех графиков
mvns = list(matrix_current['mvn'].unique())

##### Графики. Уточнить легенду!

In [10]:
for i in tqdm(mvns):
    
    print(i)
    
    try:
    
        #--------------считаем наполнение сущпол-------------------

        # выбирем данные для нужного МВН
        t = matrix_all[matrix_all['mvn'] == i].copy()
        t = t.reset_index(drop=True)
        freq = t.iloc[0]['num_of_trips']

    #------------записываем значение вместимости (за 2 часа)----------------------

        capacity = t['capacity'].values[0]

        t['Провозная способность'] = capacity
        t['idx'] = t.index

    #-----------собираем данные для графика---------------------

        cds_input = dict(x=list(t['stop_sequence'].values), # данные о входе
               top=list(t['cnt_on_base'].values/freq),
               x_offset = [x-0.2 for x in range(len(t))])
        source_input = ColumnDataSource(data = cds_input)

        cds_output = dict(x=list(t['stop_sequence'].values), # данные о выходе
               top=list(t['cnt_off_base'].values/freq),
               x_offset = [x+0.2 for x in range(len(t))])
        source_output = ColumnDataSource(data = cds_output)


        cds_input_base = dict(x=list(t['stop_sequence'].values), # данные о входе base
               top=list(t['cnt_on_current'].values/freq), 
               x_offset = [x-0.2 for x in range(len(t))])
        source_input_base = ColumnDataSource(data = cds_input_base)

        cds_output_base = dict(x=list(t['stop_sequence'].values), # данные о выходе base
               top=list(t['cnt_off_current'].values/freq), 
               x_offset = [x+0.2 for x in range(len(t))])
        source_output_base = ColumnDataSource(data = cds_output_base)    


        cds_volume = dict(x=list(t['idx'].values), # данные о наполнении
               y=list(t['cnt_inside_base'].values/freq),
               cap=list(t['Провозная способность'].values/freq),
               future = list(t['cnt_inside_current'].values/freq+0.1), 
                         )
        source_volume = ColumnDataSource(data = cds_volume)

    # ----------станции БКЛ/МЦД------------------------

        metro = list(t[t['stop_id'].isin(stops_metro)].index) # станции метро, которые надо отметить
        print(metro)

    #------------рисуем график----------------------

        p = figure(plot_width=2000, plot_height=800)

        r0 = p.vbar(x='x_offset', width=0.4, bottom=0,
           top='top', color="orange", source = source_input, )

        r1 = p.vbar(x='x_offset', width=0.4, bottom=0,
           top='top', color="royalblue", source = source_output, )

        r2 = p.vbar(x='x_offset', width=0.4, bottom=0,
           top='top', fill_color = None, line_color="red", line_dash = 'dashed',
               source = source_input_base, )

        r3 = p.vbar(x='x_offset', width=0.4, bottom=0,
           top='top', fill_color = None, line_color="purple", line_dash = 'dashed',
               source = source_output_base, )

        r4 = p.line(x='x', y='y', line_width=2, color="blue", source = source_volume, )

        p.circle(x='x', y='y', size=8, color="blue", source = source_volume)

        r5 = p.line(x='x', y='cap', line_width=2, color="red", source = source_volume, )

        r6 = p.line(x='x', y='future', line_width=2, color="limegreen", source = source_volume, )

        p.diamond(x='x', y='future', size=8, color="limegreen", source = source_volume)

        if metro:
            for m in metro:
                r7 = p.line(x=[m]*2, y = [0,0.75*capacity/freq], color = 'black',line_dash='dotted',line_width=2)

        else:
            r7 = p.line(x=0,y=0)

    #-------------легенда------------------------

        legend = Legend(items=[
            ("Посадка"   , [r0]),
            ("Высадка" , [r1]),
            ("Прогнозируемая посадка" , [r2]),
            ("Прогнозируемая высадка" , [r3]),
            ("Наполнение" , [r4]),
            ("Провозная способность" , [r5]),
            ("Прогнозируемое наполнение" , [r6]),
    #         ("Новая станция БКЛ" , [r7])
        ], location="center")

        p.add_layout(legend, 'right')

    #-------------оси---------------------------

        ax_list = list(t.index)
        p.xaxis.ticker = ax_list 

        p.xaxis.major_label_overrides = t.set_index('idx')['stop_name'].to_dict() #подписи названием
        p.xaxis.major_label_orientation = -math.pi/6 #если подписи нужно повернуть

     #-------------- форматирование подписей осей----------------  

        p.xaxis.major_label_text_font_size = '10pt'
        p.xaxis.major_label_text_align = 'center'
        p.xaxis.major_label_text_font_style = 'bold'


        p.yaxis.major_label_text_font_size = '10pt'
        p.yaxis.major_label_text_font_style = 'bold'

    #---------------устанавливаем границы графика (нужно расширить, если подписи не помещаются)

        p.min_border_right = 150
        p.min_border_left = 150

        p.title.text = str(i)
        p.title.align = "center"
        p.title.text_font_size = "25px"

        p.toolbar_location = None

    #-----------сохраняем---------------
    #   output_file(r"C:\Users\ovnanyan_gm\stages_occupancy\data\Графики загрузки\58\{0}.html".format(i))
    #   save(p)
        export_png(p, filename=r"C:\Users\ovnanyan_gm\btb_matrix\notebooks\data\Графики загрузки\194\{0}.png".format(i))
        print('saved')
        
    except:
        print('error')

  0%|          | 0/1729 [00:00<?, ?it/s]

1-00-А-обратное
[]
saved
1-00-А-прямое
[]
saved
1-00-М-обратное
[]
saved
1-00-М-прямое
[]
saved
1-00-Тм-обратное
[]
saved
1-00-Тм-прямое
[]
saved
10-00-М-обратное
[]
saved
10-00-М-прямое
[]
saved
10-00-Тм-обратное
[]
saved
10-00-Тм-прямое
[]
saved
100-00-А-обратное
[]
saved
100-00-А-прямое
[]
saved
1002-00-А-обратное
[]
saved
1002-00-А-прямое
[]
saved
1002-У2-А-прямое
[]
saved
1004-00-А-обратное
[]
saved
1004-00-А-прямое
[]
saved
102-00-А-прямое
[]
saved
103-00-А-обратное
[]
saved
103-00-А-прямое
[]
saved
104-00-А-прямое
[]
saved
104-Д1-А-прямое
[]
saved
105-00-А-прямое
[]
saved
107-00-А-прямое
[]
saved
108-00-А-обратное
[]
saved
108-00-А-прямое
[]
saved
11-00-А-обратное
[]
saved
11-00-А-прямое
[]
saved
11-00-М-обратное
[]
error
11-00-М-прямое
[]
error
11-00-Тм-обратное
[]
saved
11-00-Тм-прямое
[]
saved
11-У1-М-обратное
[]
saved
11-У1-М-прямое
[]
saved
110-00-А-обратное
[]
saved
110-00-А-прямое
[]
saved
111-00-А-прямое
[]
saved
112-00-А-обратное
[]
saved
112-00-А-прямое
[]
saved
112э-0

saved
268-00-А-прямое
[]
saved
268к-00-А-обратное
[]
saved
268к-00-А-прямое
[]
saved
27-00-Тм-обратное
[]
saved
27-00-Тм-прямое
[]
saved
270-00-А-обратное
[]
saved
270-00-А-прямое
[]
saved
271-00-А-обратное
[]
saved
271-00-А-прямое
[]
saved
272-00-А-обратное
[]
saved
272-00-А-прямое
[]
saved
273-00-А-прямое
[]
saved
276-00-А-прямое
[]
saved
278-00-А-обратное
[]
saved
278-00-А-прямое
[]
saved
279-00-А-обратное
[]
saved
279-00-А-прямое
[]
saved
28-00-Тм-обратное
[]
saved
28-00-Тм-прямое
[]
saved
281-00-А-обратное
[]
saved
281-00-А-прямое
[]
saved
282-00-А-обратное
[]
saved
282-00-А-прямое
[]
saved
283-00-А-прямое
[]
saved
283-У1-А-прямое
[]
saved
284-00-А-обратное
[]
saved
284-00-А-прямое
[]
saved
285-00-А-обратное
[]
saved
285-00-А-прямое
[]
saved
286-00-А-обратное
[]
saved
286-00-А-прямое
[]
saved
286-У1-А-обратное
[]
saved
286-У1-А-прямое
[]
saved
288-00-А-прямое
[]
saved
29-00-Тм-обратное
[]
saved
29-00-Тм-прямое
[]
saved
290-00-А-обратное
[]
saved
290-00-А-прямое
[]
saved
291-00-А-п

saved
52-00-А-прямое
[]
saved
52-У2-А-обратное
[]
saved
520-00-А-прямое
[]
saved
521-00-А-обратное
[]
saved
521-00-А-прямое
[]
saved
522-00-А-обратное
[]
saved
522-00-А-прямое
[]
saved
523-00-А-обратное
[]
saved
523-00-А-прямое
[]
saved
524-00-А-прямое
[]
saved
525-00-А-обратное
[]
saved
525-00-А-прямое
[]
saved
526-00-А-обратное
[]
saved
526-00-А-прямое
[]
saved
529-00-А-прямое
[]
saved
53-00-А-обратное
[]
saved
53-00-А-прямое
[]
saved
530-00-А-прямое
[]
saved
531-00-А-обратное
[]
saved
531-00-А-прямое
[]
saved
531-У1-А-обратное
[]
saved
531-У1-А-прямое
[]
saved
533-00-А-прямое
[]
saved
536-00-А-прямое
[]
saved
539-00-А-обратное
[]
saved
539-00-А-прямое
[]
saved
54-00-А-прямое
[]
saved
540-00-А-прямое
[]
saved
541-00-А-обратное
[]
saved
541-00-А-прямое
[]
saved
544-00-А-прямое
[]
saved
548-00-А-обратное
[]
saved
548-00-А-прямое
[]
saved
550-00-А-обратное
[]
saved
550-00-А-прямое
[]
saved
551-00-А-обратное
[]
saved
551-00-А-прямое
[]
saved
551к-00-А-прямое
[]
saved
552-00-А-прямое
[]
s

saved
750-У1-А-прямое
[]
saved
752-00-А-прямое
[]
saved
754-00-А-прямое
[]
saved
757-00-А-прямое
[]
saved
759-00-А-прямое
[]
saved
76-00-А-обратное
[]
saved
76-00-А-прямое
[]
saved
760-00-А-обратное
[]
saved
760-00-А-прямое
[]
saved
760-У1-А-обратное
[]
saved
760-У1-А-прямое
[]
saved
762-00-А-прямое
[]
saved
763-00-А-обратное
[]
saved
763-00-А-прямое
[]
saved
766-00-А-обратное
[]
saved
766-00-А-прямое
[]
saved
767-00-А-обратное
[]
saved
767-00-А-прямое
[]
saved
767-У2-А-обратное
[]
saved
769-00-А-прямое
[]
saved
769-У1-А-прямое
[]
saved
77-00-А-прямое
[]
saved
770-00-А-прямое
[]
saved
771-00-А-прямое
[]
saved
772-00-А-обратное
[]
saved
772-00-А-прямое
[]
saved
773-00-А-обратное
[]
saved
773-00-А-прямое
[]
saved
774-00-А-обратное
[]
saved
774-00-А-прямое
[]
saved
774-У5-А-обратное
[]
saved
775-00-А-обратное
[]
saved
775-00-А-прямое
[]
saved
776-00-А-обратное
[]
saved
776-00-А-прямое
[]
saved
777-У2-А-обратное
[]
saved
777-У2-А-прямое
[]
saved
777-У3-А-обратное
[]
saved
777-У3-А-прямое
[

saved
982-00-А-обратное
[]
saved
982-00-А-прямое
[]
saved
984-00-А-обратное
[]
saved
984-00-А-прямое
[]
saved
987-00-А-прямое
[]
saved
990-00-А-прямое
[]
saved
991-00-А-прямое
[]
saved
993-00-А-обратное
[]
saved
993-00-А-прямое
[]
saved
994-00-А-прямое
[]
saved
Sk1-00-А-обратное
[]
saved
Sk1-00-А-прямое
[]
saved
А-00-Тм-прямое
[]
saved
А-У1-Тм-прямое
[]
saved
Б-00-А-прямое
[]
saved
Б-У6-А-прямое
[]
saved
Б-У7-А-обратное
[]
saved
Б-У7-А-прямое
[]
saved
Бк-00-А-обратное
[]
saved
Бк-00-А-прямое
[]
saved
Бк-У1-А-обратное
[]
saved
Бк-У1-А-прямое
[]
saved
Вк-00-А-прямое
[]
saved
Вч-00-А-прямое
[]
saved
Д-00-А-обратное
[]
saved
Д-00-А-прямое
[]
saved
З-1-00-А-обратное
[]
saved
З-1-00-А-прямое
[]
saved
З-10-00-А-обратное
[]
saved
З-10-00-А-прямое
[]
saved
З-11-00-А-прямое
[]
saved
З-12-00-А-обратное
[]
saved
З-12-00-А-прямое
[]
saved
З-12-И1-А-обратное
[]
saved
З-14-00-А-обратное
[]
saved
З-14-00-А-прямое
[]
saved
З-15-00-А-обратное
[]
saved
З-15-00-А-прямое
[]
saved
З-16-00-А-обратное
[]
save

saved
с910-00-А-прямое
[]
saved
с914-00-А-обратное
[]
saved
с914-00-А-прямое
[]
saved
с916-00-А-обратное
[]
saved
с916-00-А-прямое
[]
saved
с918-00-А-прямое
[]
saved
с919-00-А-обратное
[]
saved
с919-00-А-прямое
[]
saved
с920-00-А-обратное
[]
saved
с920-00-А-прямое
[]
saved
с920-У1-А-обратное
[]
saved
с929-00-А-обратное
[]
saved
с929-00-А-прямое
[]
saved
с932-00-А-прямое
[]
saved
с935-00-А-прямое
[]
saved
с941-00-А-обратное
[]
saved
с941-00-А-прямое
[]
saved
с951-00-А-обратное
[]
saved
с951-00-А-прямое
[]
saved
с960-00-А-обратное
[]
saved
с960-00-А-прямое
[]
saved
с962-00-А-обратное
[]
saved
с962-00-А-прямое
[]
saved
с970-00-А-обратное
[]
saved
с970-00-А-прямое
[]
saved
с977-00-А-обратное
[]
saved
с977-00-А-прямое
[]
saved
с986-00-А-обратное
[]
saved
с986-00-А-прямое
[]
saved
с986-Э1-А-обратное
[]
saved
с986-Э1-А-прямое
[]
saved
с988-00-А-обратное
[]
saved
с988-00-А-прямое
[]
saved
с997-00-А-обратное
[]
saved
с997-00-А-прямое
[]
saved
т14-00-А-обратное
[]
saved
т14-00-А-прямое
[]
saved
